# 🧠 Q-Learning Algorithm: เรียนรู้จากประสบการณ์
## จาก Theory สู่ Implementation (รากฐานของ DQN)

---

## 🎯 เป้าหมายของบทนี้

จาก Notebook ที่แล้ว เรารู้แล้วว่า **Q(s,a) คืออะไร**

**คำถามสำคัญ:** 🤔
> แล้วจะหา Q-values ได้ยังไง?

**คำตอบ:** ใช้ **Q-Learning Algorithm!**

### 🚀 สิ่งที่จะเรียนรู้:

1. **Temporal Difference (TD) Learning** - เรียนรู้จากความแตกต่าง
2. **Q-Learning Update Rule** - สูตรวิเศษ
3. **Learning Rate (α)** - เรียนรู้เร็ว/ช้าแค่ไหน
4. **Off-Policy Learning** - เรียนรู้จากการกระทำใดๆ ก็ได้
5. **Implementation** - เขียนโค้ดจริง!

### 🎓 ทำไมสำคัญสำหรับ DQN?

```
Q-Learning → Deep Q-Network (DQN) → Trading Bot
    ↓              ↓                      ↓
 Q-Table      Neural Net          Portfolio Management
 (discrete)   (continuous)        (real-time trading)
```

**Q-Learning = พื้นฐานของ DQN!**
- เข้าใจ Q-Learning → เข้าใจ DQN ง่ายขึ้น
- Update rule เหมือนกัน แค่ใช้ Neural Network แทน Table

---

## 🧩 Temporal Difference (TD) Learning

### 💭 แนวคิดพื้นฐาน:

**เรียนรู้จากความผิดพลาด (Error)**

ลองนึกถึงการเรียนคณิตศาสตร์:
1. คุณคิดว่าคำตอบคือ 10 (การคาดการณ์)
2. ครูบอกว่าคำตอบที่แท้จริงคือ 15 (ความเป็นจริง)
3. คุณเรียนรู้ว่าผิดพลาด -5 (TD Error)
4. ครั้งต่อไป คุณจะปรับการคาดการณ์

### 📐 TD Error:

```
TD Error = (Actual - Predicted)
         = (Target - Q_current)
```

**ในบริบท RL:**
```
TD Error (δ) = R + γ × max Q(s',a') - Q(s,a)
                └─── Target ───┘    └─ Current ─┘
```

### 🎯 ตัวอย่างง่ายๆ:

```
Agent อยู่ที่ state A, ทำ action "Right"
→ ได้ reward = 5
→ ไปถึง state B

Current:  Q(A, Right) = 10
Target:   R + γ × max Q(B, all actions)
        = 5 + 0.9 × 20 = 23

TD Error = 23 - 10 = +13  ← คาดการณ์ต่ำไป!
```

---

## 🎯 Q-Learning Update Rule

### 📐 สูตรสำคัญที่สุด:

```python
Q(s, a) ← Q(s, a) + α × [R + γ × max Q(s', a') - Q(s, a)]
          └─ old ─┘   │   └────── TD Error ──────────┘
                      └─ Learning Rate
```

### 🔍 แยกส่วนประกอบ:

| ส่วนประกอบ | ความหมาย | ค่าทั่วไป |
|-----------|----------|----------|
| **Q(s,a)** | Q-value เดิม | - |
| **α** | Learning rate (ความเร็วในการเรียนรู้) | 0.1 - 0.5 |
| **R** | Immediate reward | ขึ้นกับ environment |
| **γ** | Discount factor | 0.9 - 0.99 |
| **max Q(s',a')** | Q-value สูงสุดของ state ถัดไป | - |

### 🎮 ตัวอย่างการคำนวณทีละขั้น:

```
สถานการณ์:
- อยู่ที่ state (0,0), ทำ action "Right"
- ได้ reward = -1 (penalty เดินแต่ละก้าว)
- ไปถึง state (0,1)
- α = 0.1, γ = 0.9

ขั้นตอน:
1. Q_current = Q(0,0, Right) = 3.0

2. หา max Q ของ state ถัดไป:
   Q((0,1), Up)    = 2.0
   Q((0,1), Down)  = 4.0  ← max
   Q((0,1), Left)  = 1.0
   Q((0,1), Right) = 5.0  ← max!
   max Q(s', a') = 5.0

3. คำนวณ Target:
   Target = R + γ × max Q(s', a')
          = -1 + 0.9 × 5.0
          = -1 + 4.5
          = 3.5

4. คำนวณ TD Error:
   TD Error = Target - Q_current
            = 3.5 - 3.0
            = 0.5

5. Update Q-value:
   Q_new = Q_old + α × TD Error
         = 3.0 + 0.1 × 0.5
         = 3.0 + 0.05
         = 3.05

✅ Q((0,0), Right) = 3.0 → 3.05
```

---

## 🎛️ Learning Rate (α): ความเร็วในการเรียนรู้

### 📊 ผลของ α ต่อการเรียนรู้:

```
Q_new = Q_old + α × TD_Error
```

### α = 0.0 (ไม่เรียนรู้เลย)
```
Q_new = Q_old + 0 × TD_Error = Q_old
❌ ไม่มีการเปลี่ยนแปลง!
```

### α = 1.0 (เชื่อข้อมูลใหม่ 100%)
```
Q_new = Q_old + 1.0 × TD_Error = Target
⚠️ ลืมประสบการณ์เก่าทันที! (unstable)
```

### α = 0.1 (ค่อนข้างช้า, มั่นคง)
```
Q_new = Q_old + 0.1 × TD_Error
✅ เรียนรู้ค่อยๆ ไป (เหมาะกับ stochastic environment)
```

### α = 0.5 (กลางๆ)
```
Q_new = Q_old + 0.5 × TD_Error
✅ เรียนรู้เร็วขึ้น (เหมาะกับ deterministic environment)
```

### 📈 การเลือก α สำหรับ Trading:

| สถานการณ์ | α ที่แนะนำ | เหตุผล |
|-----------|-----------|--------|
| **Backtest (historical data)** | 0.3 - 0.5 | ข้อมูลคงที่, เรียนรู้เร็ว |
| **Live Trading (real-time)** | 0.05 - 0.1 | ตลาดผันผวน, เรียนรู้ช้าๆ |
| **High volatility markets** | 0.01 - 0.05 | ลด noise, stable |
| **Low volatility markets** | 0.1 - 0.3 | เรียนรู้เร็วขึ้นได้ |

---

## 🔄 Off-Policy Learning: ข้อดีเด่นของ Q-Learning

### 🤔 On-Policy vs Off-Policy?

#### **On-Policy (เช่น SARSA):**
```
"เรียนรู้จาก policy ที่กำลังใช้อยู่"

Agent ทำ action a ที่ state s (ตาม policy ε-greedy)
→ Update Q โดยใช้ action ถัดไปที่ทำจริง (ตาม policy)
```

#### **Off-Policy (Q-Learning):**
```
"เรียนรู้จาก policy ใดก็ได้ → มุ่งสู่ optimal policy"

Agent ทำ action a ใดๆ ที่ state s (random/explore/exploit)
→ Update Q โดยใช้ max Q(s',a') (optimal action)
```

### 💡 ตัวอย่างเปรียบเทียบ:

```
Scenario: อยู่ที่ state S, มี 2 actions:
- Action A (safe): reward = 1, ไป state S1
- Action B (risky): reward = 10 ถ้าโชคดี, -10 ถ้าโชคร้าย

Agent explore โดยเลือก B (random)
→ โชคร้าย! ได้ reward = -10

🔴 On-Policy (SARSA):
Update Q(S,B) โดยใช้ action ที่เลือกต่อไป (อาจ random อีก)
→ เรียนรู้ช้า, มีความเสี่ยง

🟢 Off-Policy (Q-Learning):
Update Q(S,B) โดยใช้ max Q (optimal action)
→ เรียนรู้เร็ว, มุ่งสู่ optimal ทันที!
```

### 🎯 ทำไม Off-Policy ดีสำหรับ Trading?

1. **Exploration มากได้:** ลองกลยุทธ์ต่างๆ โดยไม่กระทบการเรียนรู้
2. **Reuse old data:** ใช้ historical trades ได้ (Experience Replay)
3. **Safe exploration:** เรียนรู้จากความผิดพลาดโดยไม่ติดกับ bad policy

**นี่คือรากฐานของ DQN Experience Replay!** 🚀

---

## 📋 Q-Learning Algorithm (Pseudocode)

```python
# Initialization
Initialize Q(s,a) = 0 for all s, a
Set hyperparameters: α, γ, ε

# Main Loop
for episode in range(num_episodes):
    s = env.reset()  # เริ่มต้น episode
    done = False
    
    while not done:
        # 1. เลือก action (ε-greedy)
        if random() < ε:
            a = random_action()      # Explore
        else:
            a = argmax_a Q(s, a)     # Exploit
        
        # 2. ทำ action และสังเกต result
        s', r, done = env.step(a)
        
        # 3. Update Q-value (Q-Learning Update Rule)
        Q(s,a) ← Q(s,a) + α × [r + γ × max_a' Q(s',a') - Q(s,a)]
        
        # 4. เดินต่อไป
        s = s'
    
    # (Optional) Decay ε
    ε = ε × decay_rate
```

### 🔑 Key Steps:

1. **Initialize Q-table:** เริ่มต้นด้วย 0 หรือค่าสุ่ม
2. **ε-greedy action selection:** สมดุล explore/exploit
3. **Execute action:** ทำในสภาพแวดล้อมจริง
4. **Update Q:** ใช้ Q-Learning rule
5. **Repeat:** จนกว่า converge

---

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import pandas as pd

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("📦 Libraries loaded!")
print("✅ Ready to implement Q-Learning!")

## 💻 Implementation 1: Q-Learning Update Function

In [ ]:
"""
Q-Learning Update Function

Core function ที่จะใช้ใน DQN ด้วย!
"""

def q_learning_update(Q, state, action, reward, next_state, alpha, gamma, n_actions):
    """
    Q-Learning Update Rule
    
    Parameters:
    -----------
    Q : dict or array
        Q-table {(state, action): value}
    state : tuple
        Current state
    action : int
        Action taken
    reward : float
        Reward received
    next_state : tuple
        Next state
    alpha : float
        Learning rate
    gamma : float
        Discount factor
    n_actions : int
        Number of possible actions
    
    Returns:
    --------
    td_error : float
        TD Error (for monitoring)
    """
    
    # 1. Get current Q-value
    q_current = Q.get((state, action), 0.0)
    
    # 2. Find max Q-value of next state
    max_q_next = max([Q.get((next_state, a), 0.0) for a in range(n_actions)])
    
    # 3. Calculate target
    target = reward + gamma * max_q_next
    
    # 4. Calculate TD error
    td_error = target - q_current
    
    # 5. Update Q-value
    Q[(state, action)] = q_current + alpha * td_error
    
    return td_error


# ทดสอบ function
print("🧪 Testing Q-Learning Update Function\n")
print("="*60)

# สร้าง Q-table
Q = {}

# ตั้งค่า initial Q-values
state = (0, 0)
next_state = (0, 1)
Q[(state, 3)] = 3.0  # action 3 (Right) มีค่าเริ่มต้น 3.0
Q[(next_state, 0)] = 2.0
Q[(next_state, 1)] = 4.0
Q[(next_state, 2)] = 1.0
Q[(next_state, 3)] = 5.0  # max!

print(f"Before Update:")
print(f"  Q({state}, Right) = {Q[(state, 3)]:.2f}")
print(f"\nNext State Q-values:")
print(f"  Q({next_state}, Up)    = {Q[(next_state, 0)]:.2f}")
print(f"  Q({next_state}, Down)  = {Q[(next_state, 1)]:.2f}")
print(f"  Q({next_state}, Left)  = {Q[(next_state, 2)]:.2f}")
print(f"  Q({next_state}, Right) = {Q[(next_state, 3)]:.2f} ← max!")

# ทำ update
alpha = 0.1
gamma = 0.9
reward = -1.0
action = 3  # Right
n_actions = 4

td_error = q_learning_update(Q, state, action, reward, next_state, alpha, gamma, n_actions)

print(f"\nUpdate Process:")
print(f"  Reward (R) = {reward}")
print(f"  max Q(s',a') = {max([Q[(next_state, a)] for a in range(n_actions)]):.2f}")
print(f"  Target = R + γ × max Q(s',a') = {reward} + {gamma} × 5.0 = {reward + gamma * 5.0:.2f}")
print(f"  TD Error = {td_error:.2f}")
print(f"  New Q = Old Q + α × TD Error = 3.0 + {alpha} × {td_error:.2f} = {Q[(state, action)]:.2f}")

print(f"\nAfter Update:")
print(f"  Q({state}, Right) = {Q[(state, action)]:.2f}")
print(f"\n✅ Q-value updated successfully!")

## 📊 Visualization: Learning Rate Impact

In [ ]:
"""
แสดงผลของ Learning Rate (α) ต่อการ Convergence
"""

def simulate_learning(alpha, num_updates=100):
    """
    Simulate Q-value updates with different alpha
    """
    Q_current = 0.0
    true_value = 10.0  # ค่าที่แท้จริง
    history = [Q_current]
    
    for _ in range(num_updates):
        # TD Error = True Value - Current Estimate
        td_error = true_value - Q_current
        
        # Update
        Q_current = Q_current + alpha * td_error
        history.append(Q_current)
    
    return history


# ทดลองกับ alpha ต่างๆ
alphas = [0.01, 0.1, 0.3, 0.5, 1.0]
num_updates = 50

plt.figure(figsize=(14, 6))

for alpha in alphas:
    history = simulate_learning(alpha, num_updates)
    plt.plot(history, label=f'α = {alpha}', linewidth=2, marker='o', markersize=3, alpha=0.7)

plt.axhline(y=10.0, color='red', linestyle='--', linewidth=2, label='True Value', alpha=0.5)
plt.xlabel('Number of Updates', fontsize=12, fontweight='bold')
plt.ylabel('Q-value', fontsize=12, fontweight='bold')
plt.title('Effect of Learning Rate (α) on Convergence', fontsize=14, fontweight='bold')
plt.legend(fontsize=10, loc='lower right')
plt.grid(True, alpha=0.3)
plt.ylim(-1, 12)
plt.tight_layout()
plt.show()

print("\n📊 Analysis:")
print("  - α = 0.01: เรียนรู้ช้ามาก, มั่นคง")
print("  - α = 0.1:  เรียนรู้ค่อนข้างช้า, มั่นคง (แนะนำสำหรับ Trading)")
print("  - α = 0.3:  กลางๆ, balance ดี")
print("  - α = 0.5:  เรียนรู้เร็ว")
print("  - α = 1.0:  เรียนรู้ทันที แต่ไม่มี averaging (ไม่แนะนำ)")
print("\n💡 สำหรับ Trading: ใช้ α = 0.05-0.1 เพื่อความมั่นคง")

## 🎮 Implementation 2: Simple Q-Learning Agent

In [ ]:
"""
Q-Learning Agent Class

Template นี้จะใช้ใน DQN ด้วย!
"""

class QLearningAgent:
    """
    Q-Learning Agent
    
    โครงสร้างนี้คล้ายกับ DQN Agent มาก!
    ต่างกันแค่:
    - Q-Learning ใช้ Q-table (dict)
    - DQN ใช้ Neural Network
    """
    
    def __init__(self, n_actions, alpha=0.1, gamma=0.9, epsilon=0.1):
        """
        Initialize Agent
        
        Parameters:
        -----------
        n_actions : int
            Number of possible actions
        alpha : float
            Learning rate (0-1)
        gamma : float
            Discount factor (0-1)
        epsilon : float
            Exploration rate for ε-greedy (0-1)
        """
        self.n_actions = n_actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        
        # Q-table: {(state, action): value}
        self.Q = defaultdict(float)  # default = 0.0
        
        # Statistics
        self.stats = {
            'td_errors': [],
            'q_values': [],
            'rewards': []
        }
    
    def get_q_value(self, state, action):
        """Get Q-value for (state, action)"""
        return self.Q[(state, action)]
    
    def get_max_q(self, state):
        """Get maximum Q-value for state"""
        q_values = [self.get_q_value(state, a) for a in range(self.n_actions)]
        return max(q_values)
    
    def get_best_action(self, state):
        """Get action with highest Q-value (greedy)"""
        q_values = [self.get_q_value(state, a) for a in range(self.n_actions)]
        return np.argmax(q_values)
    
    def choose_action(self, state):
        """
        Choose action using ε-greedy policy
        
        Returns:
        --------
        action : int
        """
        # ε-greedy
        if np.random.random() < self.epsilon:
            # Explore: random action
            return np.random.randint(self.n_actions)
        else:
            # Exploit: best action
            return self.get_best_action(state)
    
    def update(self, state, action, reward, next_state, done):
        """
        Q-Learning Update
        
        Parameters:
        -----------
        state : tuple
            Current state
        action : int
            Action taken
        reward : float
            Reward received
        next_state : tuple
            Next state
        done : bool
            Episode terminated?
        """
        # Current Q-value
        q_current = self.get_q_value(state, action)
        
        # Target Q-value
        if done:
            # Terminal state: no future reward
            target = reward
        else:
            # Non-terminal: R + γ × max Q(s',a')
            max_q_next = self.get_max_q(next_state)
            target = reward + self.gamma * max_q_next
        
        # TD Error
        td_error = target - q_current
        
        # Update Q-value
        self.Q[(state, action)] = q_current + self.alpha * td_error
        
        # Save statistics
        self.stats['td_errors'].append(td_error)
        self.stats['q_values'].append(q_current)
        self.stats['rewards'].append(reward)
    
    def decay_epsilon(self, decay_rate=0.995, min_epsilon=0.01):
        """Decay exploration rate"""
        self.epsilon = max(self.epsilon * decay_rate, min_epsilon)


# Test Agent
print("🤖 Q-Learning Agent Created!\n")
print("="*60)

agent = QLearningAgent(n_actions=4, alpha=0.1, gamma=0.9, epsilon=0.1)

print(f"Agent Configuration:")
print(f"  Number of actions: {agent.n_actions}")
print(f"  Learning rate (α): {agent.alpha}")
print(f"  Discount factor (γ): {agent.gamma}")
print(f"  Exploration rate (ε): {agent.epsilon}")

# ทดสอบ action selection
test_state = (0, 0)
print(f"\n🎯 Test Action Selection at state {test_state}:")

actions_selected = []
for _ in range(100):
    action = agent.choose_action(test_state)
    actions_selected.append(action)

print(f"\n  Action distribution (100 trials):")
for a in range(agent.n_actions):
    count = actions_selected.count(a)
    print(f"    Action {a}: {count}% ({'↑' if a==0 else '↓' if a==1 else '←' if a==2 else '→'})")

print(f"\n✅ Agent ready for training!")

## 🧪 Mini Experiment: 2-State MDP

In [ ]:
"""
ทดลอง Q-Learning กับ MDP ง่ายๆ

MDP:
  State A --[action: go]--> State B (reward = 0)
  State B = Terminal (reward = 10)
"""

class TwoStateMDP:
    """Simple 2-state MDP for testing"""
    
    def __init__(self):
        self.state = 'A'
    
    def reset(self):
        self.state = 'A'
        return self.state
    
    def step(self, action):
        """
        action = 0: go to B
        """
        if self.state == 'A' and action == 0:
            self.state = 'B'
            reward = 0.0
            done = False
        elif self.state == 'B':
            reward = 10.0
            done = True
        else:
            reward = -1.0
            done = False
        
        return self.state, reward, done


# Train Agent
print("🧪 Training Q-Learning on 2-State MDP\n")
print("="*60)

env = TwoStateMDP()
agent = QLearningAgent(n_actions=1, alpha=0.1, gamma=0.9, epsilon=0.0)  # no exploration needed

num_episodes = 50
q_history = []

print(f"Training for {num_episodes} episodes...\n")

for episode in range(num_episodes):
    state = env.reset()
    done = False
    
    while not done:
        action = 0  # only one action
        next_state, reward, done = env.step(action)
        agent.update(state, action, reward, next_state, done)
        state = next_state
    
    # Record Q-value
    q_a = agent.get_q_value('A', 0)
    q_history.append(q_a)
    
    if episode % 10 == 0:
        print(f"  Episode {episode:3d}: Q(A, go) = {q_a:.4f}")

print(f"\n✅ Training completed!")
print(f"\nFinal Q-values:")
print(f"  Q(A, go) = {agent.get_q_value('A', 0):.4f}")
print(f"  Q(B, -) = {agent.get_q_value('B', 0):.4f}")

# คำนวณ theoretical value
theoretical = 0 + 0.9 * 10  # R(A→B) + γ × R(B)
print(f"\n🎯 Theoretical Q(A, go) = 0 + 0.9 × 10 = {theoretical:.4f}")
print(f"📊 Learned Q(A, go) = {agent.get_q_value('A', 0):.4f}")
print(f"✅ Difference = {abs(theoretical - agent.get_q_value('A', 0)):.4f}")

# Plot convergence
plt.figure(figsize=(10, 6))
plt.plot(q_history, linewidth=2, label='Learned Q(A, go)')
plt.axhline(y=theoretical, color='red', linestyle='--', linewidth=2, label='Theoretical Value')
plt.xlabel('Episode', fontsize=12, fontweight='bold')
plt.ylabel('Q-value', fontsize=12, fontweight='bold')
plt.title('Q-Learning Convergence on 2-State MDP', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Observation:")
print("  - Q-value converges to theoretical value!")
print("  - Learning is stable with α = 0.1")
print("  - This proves Q-Learning works! 🎉")

## 🎓 เชื่อมโยงไปสู่ DQN

### 📌 Q-Learning → Deep Q-Network (DQN)

| Component | Q-Learning | DQN |
|-----------|------------|-----|
| **Q-value Storage** | Q-table (dict) | Neural Network |
| **State Space** | Discrete (small) | Continuous (large) |
| **Update Rule** | Q(s,a) ← Q(s,a) + α × TD_error | Same! Just neural net |
| **Action Selection** | argmax_a Q(s,a) | Same! |
| **Experience** | Online (forget old) | Replay Buffer (reuse) |
| **Target** | max Q(s',a') | Target Network (stable) |

### 🔄 Evolution Path:

```
1. Q-Learning (1989)
   ↓
   Q-table → Neural Network
   ↓
2. Neural Fitted Q (2005)
   ↓
   + Experience Replay
   ↓
3. DQN (2013)
   ↓
   + Target Network
   ↓
4. Double DQN (2015)
   ↓
   + Reduce overestimation
   ↓
5. Trading DQN (Now!)
   ↓
   + Portfolio management
   + Risk control
```

### 🚀 Next Steps to DQN:

**ที่เรียนแล้ว (Notebook นี้):**
- ✅ Q-Learning update rule
- ✅ TD Learning
- ✅ Off-policy learning
- ✅ Hyperparameters (α, γ, ε)

**ที่จะเรียนต่อ:**
- 📊 Grid World implementation (Notebook 03)
- 🎲 Exploration strategies (Notebook 04)
- 🧠 Neural Networks (Module 03)
- 🎮 DQN architecture (Module 04)
- 💹 Trading DQN (Module 05)

---

## 💹 Q-Learning for Trading (Preview)

### 🎯 Trading as MDP:

```python
# State
state = {
    'price': current_price,
    'position': current_position,
    'cash': available_cash,
    'indicators': [RSI, MACD, ...]  # Technical indicators
}

# Actions
actions = [0, 1, 2]  # [Buy, Hold, Sell]

# Reward
reward = PnL + penalty_for_risk

# Q-Learning Update (same!)
Q(state, action) ← Q(state, action) + α × [reward + γ × max Q(next_state, a') - Q(state, action)]
```

### 🔧 Adaptations for Trading:

1. **State Representation:**
   - Price history (OHLCV)
   - Technical indicators
   - Portfolio state
   - Market regime

2. **Action Space:**
   - Discrete: {Buy, Hold, Sell}
   - Continuous: {position_size: -1 to +1}

3. **Reward Design:**
   ```python
   reward = realized_pnl 
          - transaction_cost
          - risk_penalty (e.g., volatility)
          + diversification_bonus
   ```

4. **Hyperparameters:**
   - α = 0.001 (very small for stability)
   - γ = 0.95 - 0.99 (long-term view)
   - ε = decay from 1.0 to 0.01

---

## 🎓 สรุปบทเรียน

### 📚 สิ่งที่เรียนรู้:

#### 1️⃣ **Temporal Difference Learning**
```
TD Error = (Target - Current)
         = (R + γ × max Q(s',a')) - Q(s,a)
```

#### 2️⃣ **Q-Learning Update Rule**
```
Q(s,a) ← Q(s,a) + α × TD_Error
```

#### 3️⃣ **Learning Rate (α)**
- α ต่ำ (0.01-0.1): เรียนรู้ช้า, มั่นคง → Trading
- α สูง (0.3-0.5): เรียนรู้เร็ว → Simulation

#### 4️⃣ **Off-Policy Learning**
- เรียนรู้จาก policy ใดก็ได้
- สามารถ reuse old experiences
- รากฐานของ Experience Replay

#### 5️⃣ **Implementation**
- Q-table with defaultdict
- ε-greedy action selection
- Update function

### 🔑 Key Insights:

1. **Q-Learning = Base of DQN**
   - เข้าใจ Q-Learning → เข้าใจ DQN
   - Update rule เหมือนกัน

2. **Model-Free**
   - ไม่ต้องรู้ P(s'|s,a)
   - เหมาะกับ real-world

3. **Hyperparameters Matter**
   - α, γ, ε ต้องเลือกให้เหมาะสม
   - Trading ต้องการค่าพิเศษ

---

## 💪 แบบฝึกหัด

### Exercise 1: Manual Update
```
Q(s,a) = 5.0
Reward = 2.0
max Q(s',a') = 8.0
α = 0.1, γ = 0.9

คำนวณ Q_new = ?
```

### Exercise 2: Learning Rate
```
ทำไม Trading ควรใช้ α ต่ำ (0.05-0.1)?
ข้อดีข้อเสียคืออะไร?
```

### Exercise 3: Off-Policy
```
อธิบายว่าทำไม Off-Policy learning
ถึงเหมาะกับการใช้ Experience Replay?
```

---

## 🚀 Next Notebook

👉 **[03_q_learning_gridworld.ipynb](03_q_learning_gridworld.ipynb)**

**สิ่งที่จะเรียนรู้:**
- Implement Q-Learning ใน Grid World
- Training loop แบบสมบูรณ์
- Visualize learning progress
- Compare with Value Iteration
- Hyperparameter tuning

---

## 📚 Reference

### Papers:
- [Watkins (1989) - Q-Learning](https://link.springer.com/article/10.1007/BF00992698)
- [Mnih et al. (2013) - DQN (Atari)](https://arxiv.org/abs/1312.5602)
- [Mnih et al. (2015) - DQN (Nature)](https://www.nature.com/articles/nature14236)

### Videos:
- [David Silver RL Course - Lecture 5: Model-Free Control](https://www.youtube.com/watch?v=0g4j2k_Ggc4)
- [DeepMind x UCL - Q-Learning](https://www.youtube.com/watch?v=ISk80iLhdfU)

### Books:
- Sutton & Barto - Reinforcement Learning (Chapter 6)
- [Spinning Up in Deep RL - Q-Learning](https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html)